<a href="https://colab.research.google.com/github/tnc-br/ddf_common/blob/gen_isoscape/generate_isoscape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Variational Inference Isoscape Generation

Use this file to generate isoscapes using the variational inference model.

In [15]:
MODEL_SAVE_LOCATION = "/content/gdrive/MyDrive/amazon_rainforest_files/variational/model/kriging_numerical_stabilized/random_all_boosted_residuals.tf"
TRANSFORMER_SAVE_LOCATION = "/content/gdrive/MyDrive/amazon_rainforest_files/variational/model/kriging_numerical_stabilized/random_all_boosted_residuals_transformer.pkl"
RUN_ID = 'variational_kriging_fixed_ablated_no_val_2023_08_03'

# Only enable if you're using a good (i.e. non-hosted) machine for your runtime.
GENERATE_MAX_RESOLUTION = True


In [13]:
from osgeo import gdal, gdal_array
import numpy as np
import tensorflow as tf
from dataclasses import dataclass
from typing import List
import pandas as pd
from tqdm import tqdm
import joblib
import sys
from sklearn.compose import ColumnTransformer

!if [ ! -d "/content/ddf_common_stub" ] ; then git clone -b test https://github.com/tnc-br/ddf_common_stub.git; fi
sys.path.append("/content/ddf_common_stub/")
import ddfimport
ddfimport.ddf_import_common()

import raster

executing checkout_branch ...
Branch main already checked out.
Remember to reload your imports with `importlib.reload(module)`.
b''
main branch checked out as readonly. You may now use ddf_common imports


# Iterating over brazilian landscape

In [14]:
def get_predictions_at_each_pixel(
    model: tf.keras.Model,
    feature_transformer: ColumnTransformer,
    geotiffs: dict[str, raster.AmazonGeoTiff],
    resolution: raster.Bounds):

  # Initialize a blank plane representing means and variance.
  predicted_means_np = np.ma.array(
      np.zeros([resolution.raster_size_x, resolution.raster_size_y, 1], dtype=float),
      mask=np.ones([resolution.raster_size_x, resolution.raster_size_y, 1], dtype=bool))
  predicted_vars_np = np.ma.array(
      np.zeros([resolution.raster_size_x, resolution.raster_size_y, 1], dtype=float),
      mask=np.ones([resolution.raster_size_x, resolution.raster_size_y, 1], dtype=bool))

  for x_idx, x in enumerate(tqdm(np.arange(resolution.minx, resolution.maxx, resolution.pixel_size_x, dtype=float))):
    rows = []
    row_indexes = []
    for y_idx, y in enumerate(np.arange(resolution.miny, resolution.maxy, -resolution.pixel_size_y, dtype=float)):
      # Row should contain all the features needed to predict, in the same
      # column order the model was trained.
      row = {}
      row["lat"] = y
      row["long"] = x

      # Surround in try/except as we will be trying to fetch out of bounds data.
      try:
        for geotiff_label, geotiff in geotiffs.items():
          row[geotiff_label] = raster.get_data_at_coords(geotiff, x, y, -1)
          if pd.isnull(row[geotiff_label]):
            raise ValueError
      except (ValueError, IndexError):
        continue # masked and out-of-bounds coordinates

      rows.append(row)
      row_indexes.append((y_idx,0,))

    if (len(rows) > 0):
      X = pd.DataFrame.from_dict(rows)
      X_scaled = pd.DataFrame(feature_transformer.transform(X),
                              index=X.index, columns=X.columns)
      predictions = model.predict_on_batch(X_scaled)

      means_np = predictions[:, 0]
      for prediction, (y_idx, month_idx) in zip(means_np, row_indexes):
        predicted_means_np.mask[x_idx,y_idx,month_idx] = False
        predicted_means_np.data[x_idx,y_idx,month_idx] = prediction
      vars_np = predictions[:, 1]
      for prediction, (y_idx, month_idx) in zip (vars_np, row_indexes):
        predicted_vars_np.mask[x_idx, y_idx, month_idx] = False
        predicted_vars_np.data[x_idx, y_idx, month_idx] = prediction

  return predicted_means_np, predicted_vars_np

# Import rasters

In [ ]:
pet_geotiff = raster.load_named_raster(raster.get_raster_path("pet_Stack_mean.tiff"), "pet")
dem_geotiff = raster.load_named_raster(raster.get_raster_path("dem_pa_brasil_raster.tiff"), "dem", use_only_band_index=0)
pa_geotiff = raster.load_named_raster(raster.get_raster_path("dem_pa_brasil_raster.tiff"), "pa", use_only_band_index=1)
krig_means_geotiff = raster.load_named_raster(raster.get_raster_path("uc_davis_d18O_cel_ordinary_random_grouped_means.tiff"), "krig_means")
krig_variances_geotiff = raster.load_named_raster(raster.get_raster_path("uc_davis_d18O_cel_ordinary_random_grouped_vars.tiff"), "krig_variances")

# Note, the model inputs are order sensitive. These geotiffs should remain in this order.
# We shouldn't rely on this and we should encode the column order with the saved model.
feature_to_geotiff = {
    "VPD" : raster.vapor_pressure_deficit_geotiff(),
    "RH": raster.relative_humidity_geotiff(),
    "PET": pet_geotiff,
    "DEM": dem_geotiff,
    "PA": pa_geotiff,
    "Mean Annual Temperature" : raster.temperature_geotiff(),
    "Mean Annual Precipitation" : raster.brazil_map_geotiff(),
    "ordinary_kriging_linear_d18O_predicted_mean" : krig_means_geotiff,
    "ordinary_kriging_linear_d18O_predicted_variance" : krig_variances_geotiff
}

# Import Tensorflow model and scalers

In [ ]:
model = tf.keras.saving.load_model(MODEL_SAVE_LOCATION)
feature_transformer = joblib.load(TRANSFORMER_SAVE_LOCATION)

# Generating isoscapes

In [ ]:
resolution = raster.get_extent(pet_geotiff.gdal_dataset)

# Get the biggest resolution from the input geotiffs.
if GENERATE_MAX_RESOLUTION:
  all_bounds = [raster.get_extent(geotiff.gdal_dataset) for geotiff in feature_to_geotiff.values()]
  resolution = sorted(all_bounds, key=lambda bounds: bounds.pixel_size_x*bounds.pixel_size_y)[-1]

def generate_isoscapes_from_variational_model(
    model: tf.keras.Model,
    feature_transformer: ColumnTransformer,
    input_geotiffs: dict[str, raster.AmazonGeoTiff],
    resolution: raster.Bounds):
  means_np, vars_np = get_predictions_at_each_pixel(
    model, feature_transformer, input_geotiffs, resolution)
  raster.save_numpy_to_geotiff(
      resolution, means_np, raster.get_raster_path(RUN_ID+"_means.tiff"))
  raster.save_numpy_to_geotiff(
      resolution, vars_np, raster.get_raster_path(RUN_ID+"_vars.tiff"))

generate_isoscapes_from_variational_model(
    model, feature_transformer, feature_to_geotiff, resolution)